In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

base_dir = '/kaggle/input/eye-clasification/dataset'

label_counts = {}

for label_name in os.listdir(base_dir):
    label_path = os.path.join(base_dir, label_name)

    if os.path.isdir(label_path):
        num_images = len([f for f in os.listdir(label_path) if os.path.isfile(os.path.join(label_path, f))])
        label_counts[label_name] = num_images

df = pd.DataFrame(list(label_counts.items()), columns=['Label', 'Count'])

labels = df['Label'].tolist()
counts = df['Count'].tolist()

colors = plt.cm.viridis(np.linspace(0, 1, len(labels)))

plt.figure(figsize=(10, 6))
bars = plt.bar(labels, counts, color=colors)

plt.xlabel('Labels')
plt.ylabel('Number of Images')
plt.title('Distribution of Eye Diseases')
plt.xticks(rotation=45, ha='right')

for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() / 2, yval, int(yval), ha='center', va='bottom')

plt.tight_layout()
plt.show()


In [ ]:
import os
import random
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import math

# Definisi fungsi untuk menampilkan sample gambar
def display_sample_img(data_dir, title):
    # Ambil daftar label, kecuali 'train' dan 'test' jika ada
    labels = [label for label in os.listdir(data_dir)
              if os.path.isdir(os.path.join(data_dir, label))
              and label not in ['train', 'test']]

    num_labels = len(labels)
    cols = math.ceil(num_labels / 2)
    rows = 2 if num_labels > 1 else 1

    plt.figure(figsize=(15, 6))
    plt.suptitle(title, fontsize=16)

    for idx, label in enumerate(labels):
        label_path = os.path.join(data_dir, label)
        image_files = [f for f in os.listdir(label_path)
                       if os.path.isfile(os.path.join(label_path, f))]

        if image_files:
            random_image = random.choice(image_files)
            img_path = os.path.join(label_path, random_image)
            img = mpimg.imread(img_path)

            plt.subplot(rows, cols, idx + 1)
            plt.imshow(img)
            plt.axis('off')
            plt.title(label)

    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.show()

# Path dataset Kaggle
data_path = '/kaggle/input/eye-clasification/dataset'

# Pengecekan apakah folder tersedia
if os.path.exists(data_path):
    display_sample_img(data_path, 'Sample Images of Eye Diseases')
else:
    print(f"Dataset folder not found at: {data_path}")


In [ ]:
dataset_dir = '/kaggle/input/eye-clasification/dataset'
desired_labels = ['cataract', 'diabetic_retinopathy', 'glaucoma', 'normal']

file_name = []
labels = []
full_path = []

for path, subdirs, files in os.walk(dataset_dir):
  for name in files:
    label = os.path.basename(path)
    if label in desired_labels:
      full_path.append(os.path.join(path, name))
      labels.append(label)
      file_name.append(name)

df = pd.DataFrame({"path": full_path, "file_name": file_name, "labels": labels})
print(df.groupby(['labels']).size())

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Misalnya df sudah berisi kolom 'path' dan 'labels'
# Contoh minimal isi df:
# df = pd.DataFrame({
#     'path': ['/path/to/image1.jpg', '/path/to/image2.jpg', ...],
#     'labels': ['Cataract', 'Glaucoma', ...]
# })

# Ekstrak fitur dan label
X = df['path']
y = df['labels']

# Split menjadi training dan testing set
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Buat DataFrame baru untuk train dan test
df_train = pd.DataFrame({'path': X_train, 'labels': y_train, 'set': 'train'})
df_test = pd.DataFrame({'path': X_test, 'labels': y_test, 'set': 'test'})

# Gabungkan agar menjadi satu DataFrame dengan kolom 'set' sebagai penanda
df_split = pd.concat([df_train, df_test]).reset_index(drop=True)

# (Opsional) Lihat hasil
print(df_split.head())
print('train size', len(df_train))
print('test size', len(df_test))

In [ ]:
print('train size', len(df_train))
print('test size', len(df_test))

In [ ]:
import os
import shutil  # ← ini penting!
import pandas as pd
from sklearn.model_selection import train_test_split

# Contoh: df sudah ada dengan kolom 'path' dan 'labels'
# df = pd.DataFrame({
#     'path': ['/content/eye-diseases/dataset/Cataract/img1.jpg', ...],
#     'labels': ['Cataract', ...]
# })

# Split train/test
X = df['path']
y = df['labels']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

df_train = pd.DataFrame({'path': X_train, 'labels': y_train, 'set': 'train'})
df_test = pd.DataFrame({'path': X_test, 'labels': y_test, 'set': 'test'})

# Path sumber dan target
datasource_path = "/content/eye-diseases/dataset"
dataset_path = "/content/eye-diseases/dataset"

# Fungsi untuk menyalin file
def move_files(df, source_path, target_path):
    for index, row in df.iterrows():
        file_path = row['path']  # path absolut
        label = row['labels']
        file_set = row['set']

        dest_dir = os.path.join(target_path, file_set, label)
        os.makedirs(dest_dir, exist_ok=True)

        destination_file_name = os.path.basename(file_path)
        file_dest = os.path.join(dest_dir, destination_file_name)

        if not os.path.exists(file_dest):
            shutil.copy2(file_path, file_dest)

# Eksekusi pemindahan file
move_files(df_train, datasource_path, dataset_path)
move_files(df_test, datasource_path, dataset_path)

print("The data has been moved and separated into train and test.")


In [ ]:
TRAIN_DIR = '/content/eye-diseases/dataset/train'
TEST_DIR = '/content/eye-diseases/dataset/test'

train_normal = os.path.join(TRAIN_DIR, 'normal')
train_diabetic_retinopathy = os.path.join(TRAIN_DIR, 'diabetic_retinopathy')
train_glaucoma = os.path.join(TRAIN_DIR, 'glaucoma')
train_cataract = os.path.join(TRAIN_DIR, 'cataract')

test_normal = os.path.join(TEST_DIR, 'normal')
test_diabetic_retinopathy = os.path.join(TEST_DIR, 'diabetic_retinopathy')
test_glaucoma = os.path.join(TEST_DIR, 'glaucoma')
test_cataract = os.path.join(TEST_DIR, 'cataract')

def count_images(directory):
    return len(os.listdir(directory))

train_counts = {
    'Normal': count_images(train_normal),
    'Diabetic Retinopathy': count_images(train_diabetic_retinopathy),
    'Glaucoma': count_images(train_glaucoma),
    'Cataract': count_images(train_cataract),
}

test_counts = {
    'Normal': count_images(test_normal),
    'Diabetic Retinopathy': count_images(test_diabetic_retinopathy),
    'Glaucoma': count_images(test_glaucoma),
    'Cataract': count_images(test_cataract),
}

print("Training Image Counts:", train_counts)
print("Testing Image Counts:", test_counts)

In [ ]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Konstanta
BATCH_SIZE = 32
IMG_SIZE = (224, 224)

# Direktori data
TRAIN_DIR = '/content/eye-diseases/dataset/train'
TEST_DIR = '/content/eye-diseases/dataset/test'

# Augmentasi data training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    horizontal_flip=True,
    zoom_range=0.1,
    brightness_range=[0.8, 1.2],
    fill_mode='nearest',
)

# Hanya rescale untuk data testing (tanpa augmentasi)
test_datagen = ImageDataGenerator(rescale=1./255)

# Data generator training
train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    color_mode="rgb",
    shuffle=True,
)

# Data generator testing
test_generator = test_datagen.flow_from_directory(
    TEST_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    color_mode="rgb",
    shuffle=False,
)


In [ ]:
print("classes: ", train_generator.class_indices)

In [ ]:
input_shape = (224, 224, 3)
classes = 4

base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=input_shape)
base_model.trainable = True

model = Sequential([
    base_model,
    BatchNormalization(),
    Conv2D(512, (3, 3), activation='relu', padding='same'),
    GlobalAveragePooling2D(),
    Dense(256, activation='relu'),
    BatchNormalization(),
    Dropout(0.4),
    Dense(classes, activation='softmax')
])

optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3, clipnorm=1.0)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
def cyclic_lr(epoch, lr):
    base_lr = 1e-5
    max_lr = 1e-3
    step_size = 2000
    cycle = np.floor(1 + epoch / (2 * step_size))
    x = np.abs(epoch / step_size - 2 * cycle + 1)
    lr = base_lr + (max_lr - base_lr) * max(0, (1 - x))
    return lr

In [ ]:
class stopTraining(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    val_accuracy = logs.get('val_accuracy')
    val_loss = logs.get('val_loss')

    if val_accuracy is not None and val_loss is not None:
      if val_accuracy >= 0.93 and val_loss < 0.13:
        print('Akurasi di atas 93%, stop training')
        self.model.stop_training = True

stop_train = stopTraining()

In [ ]:
callbacks = [
    stop_train,
    LearningRateScheduler(cyclic_lr),
    EarlyStopping(monitor='val_loss', patience=30, restore_best_weights=True),
    ModelCheckpoint(filepath='best_model_1.keras', monitor='val_loss', save_best_only=True, verbose=1, mode='min')
]

In [ ]:
count_normal = len(os.listdir(train_normal))
count_diabetic_retinopathy = len(os.listdir(train_diabetic_retinopathy))
count_glaucoma = len(os.listdir(train_glaucoma))
count_cataract = len(os.listdir(train_cataract))

total_images = (count_normal + count_diabetic_retinopathy + count_glaucoma + count_cataract)

weight_normal = (1 / count_normal) * total_images / 4
weight_diabetic_retinopathy = (1 / count_diabetic_retinopathy) * total_images / 4
weight_glaucoma = (1 / count_glaucoma) * total_images / 4
weight_cataract = (1 / count_cataract) * total_images / 4

class_weights = {
    0: weight_cataract,
    1: weight_diabetic_retinopathy,
    2: weight_glaucoma,
    3: weight_normal
}

print("Class Weights:", class_weights)


In [ ]:
history = model.fit(train_generator,
                    epochs = 106,
                    validation_data=test_generator,
                    class_weight = class_weights,
                    callbacks=callbacks
                  )

In [ ]:
test_generator.reset()

train_acc = history.history['accuracy'][-1]
val_acc = history.history['val_accuracy'][-1]
test_loss, test_accuracy = model.evaluate(test_generator)

print('Modev Evaluation')
print(f"Final Training Accuracy: {train_acc:.2f}")
print(f"Final Validation Accuracy: {val_acc:.2f}")
print(f'Test Loss: {test_loss:.2f}, Test Accuracy: {test_accuracy:.2f}')

In [ ]:
plt.figure(figsize=(10,6))
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(10,6))
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.grid(True)
plt.legend(loc='upper right')
plt.show()

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report

# Pastikan test_generator masih aktif
test_generator.reset()

# Prediksi
preds_1 = model.predict(test_generator, verbose=0)
preds_1 = preds_1.argmax(axis=1)

# Daftar label (urutannya harus sesuai urutan dari test_generator.class_indices)
labels = ['cataract', 'diabetic_retinopathy', 'glaucoma', 'normal']

# Confusion matrix
cm = pd.DataFrame(
    data=confusion_matrix(test_generator.classes, preds_1, labels=range(len(labels))),
    index=[f"Actual {label}" for label in labels],
    columns=[f"Predicted {label}" for label in labels]
)

# Visualisasi heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.title("Confusion Matrix")
plt.show()

# Classification report
print("\nClassification Report:\n")
print(classification_report(
    y_true=test_generator.classes,
    y_pred=preds_1,
    target_names=labels,
    digits=4
))


In [ ]:
from sklearn.metrics import roc_curve, auc, balanced_accuracy_score

# Hitung balanced accuracy
balanced_acc = balanced_accuracy_score(test_generator.classes, preds_1)
print(f"Balanced Accuracy: {balanced_acc:.4f}")

# ROC Curve untuk setiap kelas
plt.figure(figsize=(12, 8))
for i, label in enumerate(labels):
    fpr, tpr, _ = roc_curve(test_generator.classes == i, preds_1 == i)
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, label=f'{label} (AUC = {roc_auc:.2f})')

plt.title('ROC Curve per Class')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend()
plt.grid()
plt.show()


In [ ]:
saved_model_path = './saved_model_1/eye_diseases_model_1'
model.export(saved_model_path)
print(f"Model disimpan dalam format SavedModel di: {saved_model_path}")

In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_path)
tflite_model = converter.convert()

tflite_model_dir = './saved_model_1/tflite_model/'
os.makedirs(tflite_model_dir, exist_ok=True)

converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

tflite_model_path = os.path.join(tflite_model_dir, 'eye_diseases_model_1.tflite')
with open(tflite_model_path, 'wb') as f:
    f.write(tflite_model)

labels = ['cataract', 'diabetic_retinopathy', 'glaucoma', 'normal']

label_file_path = os.path.join(tflite_model_dir, 'label.txt')
with open(label_file_path, 'w') as f:
    for label in labels:
        f.write(f"{label}\n")

print(f"Model disimpan dalam format TF-Lite di: {tflite_model_path}")
print(f"Labels disimpan dalam file: {label_file_path}")

In [ ]:
!pip install tensorflowjs

In [ ]:
saved_model_path = './saved_model_1/saved_model/'  # Contoh path
# Pastikan model sudah tersimpan di lokasi ini


In [ ]:
saved_model_path = './saved_model_1/saved_model/'  # Contoh path
# Pastikan model sudah tersimpan di lokasi ini


In [ ]:
import tensorflow as tf

# Memuat model TFLite
interpreter = tf.lite.Interpreter(model_path='./saved_model_1/tflite_model/eye_diseases_model_1.tflite')
interpreter.allocate_tensors()

# Mendapatkan informasi tensor
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print("Input details:", input_details)
print("Output details:", output_details)


In [ ]:
import os
import tensorflowjs as tfjs

tfjs_model_dir = './saved_model_1/tfjs_model/'
os.makedirs(tfjs_model_dir, exist_ok=True)

tfjs_model_path = os.path.join(tfjs_model_dir, 'model')
tfjs.converters.save_keras_model(model, tfjs_model_path)


In [ ]:
folder_to_zip = './saved_model_1'
zip_file_path = './eye_saved_model_1.zip'

shutil.make_archive(base_name=zip_file_path.replace('.zip', ''), format='zip', root_dir=os.path.dirname(folder_to_zip), base_dir=os.path.basename(folder_to_zip))

print(f"Folder '{folder_to_zip}' telah dikompres menjadi '{zip_file_path}'.")

In [ ]:
# Kompres folder
import shutil
import os

folder_to_zip = './saved_model_1'
zip_file_path = './eye_saved_model_1.zip'

shutil.make_archive(
    base_name=zip_file_path.replace('.zip', ''),
    format='zip',
    root_dir=os.path.dirname(folder_to_zip),
    base_dir=os.path.basename(folder_to_zip)
)

print(f"Folder '{folder_to_zip}' telah dikompres menjadi '{zip_file_path}'.")


In [ ]:
!ls -lh ./eye_saved_model_1.zip


In [ ]:
import zipfile
import os

zip_file_path = 'eye_saved_model_1.zip'
output_folder = 'eye_saved_model'

os.makedirs(output_folder, exist_ok=True)

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(output_folder)

print(f"File telah diekstrak ke folder: {output_folder}")

In [ ]:
def predict_images(folder_path):
    model_path = './eye_saved_model/saved_model_1/eye_diseases_model_1'
    model = tf.saved_model.load(model_path)
    infer = model.signatures['serving_default']

    labels = ['cataract', 'diabetic_retinopathy', 'glaucoma', 'normal']
    custom_labels = {
        'c': 'cataract',
        'dr': 'diabetic retinopathy',
        'h': 'healthy',
        'g': 'glaucoma'
    }

    images = []
    titles = []
    actual_labels = []

    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
            image = Image.open(file_path)
            image = image.resize((224, 224))
            image_array = np.array(image).astype(np.float32) / 255.0
            image_array = np.expand_dims(image_array, axis=0)

            predictions = infer(tf.convert_to_tensor(image_array, dtype=tf.float32))['output_0']
            predicted_index = np.argmax(predictions.numpy(), axis=1)[0]
            predicted_probability = np.max(predictions.numpy()) * 100
            predicted_label = labels[predicted_index]

            actual_label = filename.split('_')[-1].split('.')[0]
            actual_label = custom_labels.get(actual_label, actual_label)

            title = f"actual: {actual_label}\n predicted: {predicted_label} ({predicted_probability:.2f}%)"
            titles.append(title)
            images.append(image)

    num_images = len(images)
    num_columns = 3
    num_rows = (num_images + num_columns - 1) // num_columns

    plt.figure(figsize=(13, num_rows * 5))

    for i in range(num_images):
        plt.subplot(num_rows, num_columns, i + 1)
        plt.imshow(images[i])
        plt.axis('off')
        plt.title(titles[i])

    plt.suptitle("Prediction", y=0.85)
    plt.subplots_adjust(top=0.75, wspace=0.3, hspace=0.4)
    plt.show()

In [ ]:
import os
import shutil
import zipfile

folder_path = '/kaggle/working/new_eye_data'

if os.path.exists(folder_path):
    shutil.rmtree(folder_path)
    print(f"Folder '{folder_path}' telah dihapus.")
else:
    print(f"Folder '{folder_path}' tidak ditemukan.")

# Ganti sesuai nama file yang ada
zip_file_path = '/kaggle/working/eye_saved_model_1.zip'
extract_to = '/kaggle/working/new_eye_data'

os.makedirs(extract_to, exist_ok=True)

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

print(f"File diekstrak ke folder: {extract_to}")


In [ ]:
!ls -R /kaggle/working/new_eye_data


In [ ]:
folder_path = '/kaggle/working/new_eye_data/test_images'
predict_images(folder_path)
